In [2]:
import numpy as np 
import pandas as pd 
from pandas_profiling import ProfileReport

In [9]:
filepath = "../input/scrabble-player-rating"
df_train = pd.read_csv(os.path.join(filepath, "train.csv"))
df_turns = pd.read_csv(os.path.join(filepath, "turns.csv"))
df_games = pd.read_csv(os.path.join(filepath, "games.csv"))
df_sub = pd.read_csv(os.path.join(filepath, "sample_submission.csv"))

In [21]:
display(df_train.head())
display(df_turns.head())
display(df_games.head())
display(df_sub.head())

,game_id,nickname,score,rating
0,1,BetterBot,335,1637
1,1,stevy,429,1500
2,3,davidavid,440,1811
3,3,BetterBot,318,2071
4,4,Inandoutworker,119,1473


,game_id,turn_number,nickname,rack,location,move,points,score,turn_type
0,1,1,BetterBot,DDEGITT,8G,DIG,10,10,Play
1,1,2,stevy,AEHOPUX,7H,HAP,18,18,Play
2,1,3,BetterBot,DEELTTU,6I,LUTE,16,26,Play
3,1,4,stevy,EMORSUX,5K,UM,16,34,Play
4,1,5,BetterBot,ACDEITU,L5,..DICATE,28,54,Play


,game_id,first,time_control_name,game_end_reason,winner,created_at,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds
0,1,BetterBot,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,674.844274
1,2,Super,regular,STANDARD,1,2022-08-10 19:19:59,CSW21,3600,0,RATED,1,364.214418
2,3,BetterBot,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,492.268262
3,4,BetterBot,regular,RESIGNED,0,2022-09-12 02:36:19,CSW21,3600,0,CASUAL,1,350.861141
4,5,STEEBot,regular,STANDARD,0,2022-09-06 04:31:36,NWL20,1200,0,CASUAL,1,642.688722


,game_id,rating
0,2,1790.322674
1,7,1790.322674
2,11,1790.322674
3,14,1790.322674
4,27,1790.322674


In [16]:
display(df_train.describe())
display(df_turns.describe())
display(df_games.describe())

,game_id,score,rating
count,100820.000000,100820.000000,100820.000000
mean,36367.328605,399.029022,1875.592938
std,21020.288705,75.184778,231.791016
min,1.000000,-64.000000,1033.000000
25%,18139.000000,352.000000,1664.000000
50%,36339.500000,398.000000,1907.000000
75%,54625.000000,445.000000,2060.000000
max,72773.000000,1132.000000,2510.000000


,game_id,turn_number,points,score
count,2.005498e+06,2.005498e+06,2.005498e+06,2.005498e+06
mean,3.640032e+04,1.453798e+01,2.900326e+01,2.270914e+02
std,2.100559e+04,8.434285e+00,2.210145e+01,1.343110e+02
min,1.000000e+00,1.000000e+00,-2.210000e+02,-6.400000e+01
25%,1.820800e+04,7.000000e+00,1.600000e+01,1.130000e+02
50%,3.640100e+04,1.400000e+01,2.400000e+01,2.240000e+02
75%,5.460900e+04,2.100000e+01,3.600000e+01,3.320000e+02
max,7.277300e+04,7.500000e+01,3.110000e+02,1.171000e+03


,game_id,winner,initial_time_seconds,increment_seconds,max_overtime_minutes,game_duration_seconds
count,72773.000000,72773.000000,72773.000000,72773.00000,72773.000000,72773.000000
mean,36387.000000,0.350734,1198.034230,0.22856,2.143707,484.069347
std,21007.899907,0.484859,713.185219,2.54802,2.989853,333.563128
min,1.000000,-1.000000,15.000000,0.00000,0.000000,7.962989
25%,18194.000000,0.000000,900.000000,0.00000,1.000000,250.172459
50%,36387.000000,0.000000,1200.000000,0.00000,1.000000,399.834150
75%,54580.000000,1.000000,1200.000000,0.00000,1.000000,633.424285
max,72773.000000,1.000000,3600.000000,60.00000,10.000000,4444.821645


- rack(手持ちのアルファベット)とlocation(どこに置いたか)で欠損がある
  - 駒を置けない場合NA?

In [18]:
display(df_train.isna().sum())
display(df_turns.isna().sum())
display(df_games.isna().sum())

game_id     0
nickname    0
score       0
rating      0
dtype: int64

game_id             0
turn_number         0
nickname            0
rack            69390
location       132239
move              454
points              0
score               0
turn_type         395
dtype: int64

game_id                  0
first                    0
time_control_name        0
game_end_reason          0
winner                   0
created_at               0
lexicon                  0
initial_time_seconds     0
increment_seconds        0
rating_mode              0
max_overtime_minutes     0
game_duration_seconds    0
dtype: int64

In [20]:
df_turns[df_turns["rack"].isna()].head()

,game_id,turn_number,nickname,rack,location,move,points,score,turn_type
26,1,27,stevy,NaN,NaN,(O),2,429,End
52,2,26,Super,NaN,NaN,(T),2,488,End
79,3,27,davidavid,NaN,NaN,(IQRRS),28,440,End
139,5,31,stevy,NaN,NaN,(NU),4,325,End
164,6,25,STEEBot,NaN,NaN,(BIU),10,427,End
